# Standing up EJFAT LB on a FABRIC U280-equipped node

Your compute nodes can include FPGAs. These devices are made available as FABRIC components and can be added to your nodes like any other component. Your project must have Component.FPGA permission tag in order to be able to provision them. 

This notebook stands up a single VM with an attached U280 and starts up the necessary stacks on it to get the Load Balancer running. There is an optional set of steps in the middle to help build the needed docker containers. Alternatively they can be fetched from a storage VM on EDC (there is another notebook that shows how to stand it up - it is attached to a persistent storage volume that contains pre-built artifacts).


## Setup the Experiment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

# until fablib fixes this
def get_management_os_interface(node) -> str or None:
        """
        Gets the name of the management interface used by the node's
        operating system. 

        :return: interface name
        :rtype: String
        """
        stdout, stderr = node.execute("sudo ip -j route list", quiet=True)
        stdout_json = json.loads(stdout)

        for i in stdout_json:
            if i["dst"] == "default":
                return i["dev"]

        stdout, stderr = node.execute("sudo ip -6 -j route list", quiet=True)
        stdout_json = json.loads(stdout)

        for i in stdout_json:
            if i["dst"] == "default":
                return i["dev"]

        return None

def execute_single_node(node, commands):
    for command in commands:
        print(f'\tExecuting "{command}" on node {node.get_name()}')
        #stdout, stderr = node.execute(command, quiet=True, output_file=node.get_name() + '_install.log')
        stdout, stderr = node.execute(command)
    if not stderr and len(stderr) > 0:
        print(f'Error encountered with "{command}": {stderr}')
        
def execute_commands(node, commands):
    if isinstance(node, list):
        for n in node:
            execute_single_node(n, commands)
    else:
        execute_single_node(node, commands)

def execute_single_node_on_thread(node, commands):
    # concatenate the commands using ';' and execute
    allcommands = ';'.join(commands)
    node.execute_thread(allcommands, output_file=node.get_name() + '_thread.log')

def execute_commands_on_threads(node, commands):
    if isinstance(node, list):
        for n in node:
            execute_single_node_on_thread(n, commands)
    else:
        execute_single_node_on_thread(node, commands)

## Select a site with E2SAR-assigned FPGA

The cells below help you create a slice that contains a single node with an attached FPGA. 

In [ ]:
# FPGA site should only be one of these as these are assigned to the project
# 'WASH', 'KANS', or 'LOSA'
site='LOSA'

FPGA_CHOICE='FPGA_Xilinx_U280'

# name the slice and the node 
slice_name=f'E2SAR U280 LB Slice on {site}'

fpga_node_name='LB-node'
image = 'default_ubuntu_22'
net_name = '_'.join(['fabnetv4ext', site])

# storage VM - update this after you provision it using the other notebook
storage_vm_ip = "10.132.133.2"
nginx_user = "fpga_tools"
nginx_password = "changemenow1"

# version to use for saving docker images
docker_image_version = "11142024"

print(f'Will create slice "{slice_name}" with node "{fpga_node_name}"')

## Create a slice with a node with FPGA at desired site

This slice has two VMs - one with the FPGA and the other with a ConnectX-6 card - we will want to pass traffic between them.

In [ ]:
# Create Slice. Note that by default submit() call will poll for 360 seconds every 10-20 seconds
# waiting for slice to come up. Normal expected time is around 2 minutes. 
slice = fablib.new_slice(name=slice_name)

# Add node with a 100G drive and 8 of CPU cores using Ubuntu 22 image
node = slice.add_node(name=fpga_node_name, site=site, cores=8, ram=24, disk=500, image=image)

# postboot configuration is under 'post-boot' directory
node.add_post_boot_upload_directory('post-boot','.')
node.add_post_boot_execute(f'chmod +x post-boot/lb-node.sh && ./post-boot/lb-node.sh')
# FABNetv4 on shared NIC (to talk to storage)
node.add_fabnet()

fpga_comp = node.add_component(model=FPGA_CHOICE, name='fpga1')
fpga_p1 = fpga_comp.get_interfaces()[0]
fpga_p2 = fpga_comp.get_interfaces()[1]

# use FABNetv4Ext to connect port 1
net = slice.add_l3network(name=net_name, interfaces=[fpga_p1], type='IPv4Ext')

# Submit Slice Request
slice.submit();

In [ ]:
# Get slice details. You can rerun this as many times as you want
slice = fablib.get_slice(name=slice_name)
node = slice.get_node(name=fpga_node_name)

## Setup IOMMU and Hugepages
For DPDK to function properly we need to setup hugepages and IOMMU on the VM

In [ ]:
slice = fablib.get_slice(name=slice_name)
node = slice.get_node(name=fpga_node_name)

commands = list()
#commands.append("sudo sed -i 's/GRUB_CMDLINE_LINUX=\"\\(.*\\)\"/GRUB_CMDLINE_LINUX=\"\\1 amd_iommu=on iommu=pt default_hugepagesz=1G hugepagesz=1G hugepages=8\"/' /etc/default/grub")
commands.append("sudo sed -i 's/GRUB_CMDLINE_LINUX=\"\"/GRUB_CMDLINE_LINUX=\"amd_iommu=on iommu=pt default_hugepagesz=1G hugepagesz=1G hugepages=8\"/' /etc/default/grub")
commands.append("sudo grub-mkconfig -o /boot/grub/grub.cfg")
commands.append("sudo update-grub")

for command in commands:
    print(f'Executing {command}')
    stdout, stderr = node.execute(command)
    
print('Done')

Reboot the node (this sometimes generates an EOFError exception - ignore it and continue)

In [ ]:
reboot = 'sudo reboot'

print(reboot)
node.execute(reboot)

slice.wait_ssh(timeout=360,interval=10,progress=True)

print("Now testing SSH abilites to reconnect...",end="")
slice.update()
slice.test_ssh()
print("Reconnected!")

Check that IOMMU was enabled

In [ ]:
command = 'sudo dmesg | grep -i IOMMU'

print('Observe that the modifications to boot configuration took place and IOMMU is detected')
stdout, stderr = node.execute(command)

node.config()

Disable IOMMU support in VFIO (the passing through doesn't actually work)

In [ ]:
# Enable unsafe_noiommu_mode for the vfio module
command = "echo 1 | sudo tee /sys/module/vfio/parameters/enable_unsafe_noiommu_mode"

stdout, stderr = node.execute(command)

## Update Docker daemon configuration to make sure builds work on IPv6 hosts

In [ ]:
node.upload_file('config/daemon.json', 'daemon.json')
commands = [
    "sudo mv daemon.json /etc/docker/; sudo chown root:root /etc/docker/daemon.json",
    "sudo systemctl restart docker",
    "sudo systemctl status docker"
]

execute_commands(node, commands)

## Build the docker containers and upload to storage VM (optional)

We have to build 3 container images:
- xilinx-labtools-docker (requires Xilinx labtools)
- smartnic-dpdk-docker 
- esnet-smartnic-fw (requires P4 artifact from ESnet team)

These are `docker compose`d together into a running stack. In addition UDPLBd must be run on top of this stack to provide the control plane functionality.

### Building xilinx-labtools-docker container

Following these instructions:
- https://github.com/esnet/xilinx-labtools-docker/

Overall steps are:
- Checkout the repo https://github.com/esnet/xilinx-labtools-docker/
- Download the missing files from Storage VM
- Run docker build

In [ ]:
# this is a map between destination directory names and files. On the storage VM all files are in the same directory"
fetch_file_list = {
    "sc-fw-downloads": ["SC_U280_4_3_31.zip", "SC_U55C_7_1_23.zip", "loadsc_v2.3.zip"],
    "vivado-installer": ["Vivado_Lab_Lin_2023.2_1013_2256.tar.gz"]
}
commands = [
    f"git clone https://github.com/esnet/xilinx-labtools-docker.git"
]

curl_command = f"curl -s -k -u {nginx_user}:{nginx_password} https://{storage_vm_ip}/ejfat-data/artifacts/Vivado-Labtools/"

for ddir, filelist in fetch_file_list.items():
    for file in filelist:
        commands.append(f"{curl_command}{file} > {file}")
        commands.append(f"mv {file} xilinx-labtools-docker/{ddir}")
          
execute_commands(node, commands)

In [ ]:
# build step

commands = [
    "cd xilinx-labtools-docker; docker build --pull -t xilinx-labtools-docker:${USER}-dev .",
    "docker image ls"
]
execute_commands(node, commands)

In [ ]:
# save image to file and upload to the storage VM
commands = [
    f"docker save xilinx-labtools-docker | gzip > xilinx-labtools-docker-{docker_image_version}.tar.gz",
    f"curl -s -k -u {nginx_user}:{nginx_password} -T xilinx-labtools-docker-{docker_image_version}.tar.gz https://{storage_vm_ip}/ejfat-data/smartnic-docker-images/"
]

execute_commands(node, commands)

### Building smartnic-dpdk-docker container

Following these instructions
- https://github.com/esnet/smartnic-dpdk-docker

In [ ]:
# clone
commands = [
    "git clone https://github.com/esnet/smartnic-dpdk-docker.git",
    "cd smartnic-dpdk-docker; git submodule update --init --recursive"
]

execute_commands(node, commands)

In [ ]:
# build step

commands = [
    "cd smartnic-dpdk-docker; docker build --pull -t smartnic-dpdk-docker:${USER}-dev .",
    "docker image ls"
]

execute_commands(node, commands)

In [ ]:
# save image to file and upload to the storage VM
commands = [
    f"docker save smartnic-dpdk-docker | gzip > smartnic-dpdk-docker-{docker_image_version}.tar.gz",
    f"curl -s -k -u {nginx_user}:{nginx_password} -T smartnic-dpdk-docker-{docker_image_version}.tar.gz https://{storage_vm_ip}/ejfat-data/smartnic-docker-images/"
]

execute_commands(node, commands)

### Building esnet-smartnic-fw container

Following these instructions:
- https://github.com/esnet/esnet-smartnic-fw/blob/main/sn-stack/README.INSTALL.md

In [ ]:
# clone. You need to do this step even if you've built this before, sn-stack/ gets untarred into this.
commands = [
    "rm -rf esnet-smartnic-fw",
    "git clone https://github.com/esnet/esnet-smartnic-fw.git",
    "cd esnet-smartnic-fw; git submodule init && git submodule update"
]

execute_commands(node, commands)

In [ ]:
# Download the P4 artifact and install in the right place

p4_artifact = "artifacts.au280.udplb.57684.zip"

commands = [
    f"curl -s -k -u {nginx_user}:{nginx_password} https://{storage_vm_ip}/ejfat-data/artifacts/P4/{p4_artifact} > {p4_artifact}",
    f"mv {p4_artifact} esnet-smartnic-fw/sn-hw"
]

execute_commands(node, commands)

In [ ]:
# create an env file for the build
# if the P4 bitfile was called artifacts.au280.udplb.57684.zip, then
# SN_HW_VER=57684
# SN_HW_BOARD=au280 
# SN_HW_APP_NAME=udplb

env_file = """
SN_HW_VER=57684
SN_HW_BOARD=au280
SN_HW_APP_NAME=udplb
"""

commands = [
    f"cd esnet-smartnic-fw; rm -f .env; cp example.env .env",
    f"echo '{env_file}' >> ~/esnet-smartnic-fw/.env"
]
execute_commands(node, commands)

In [ ]:
# run the build

commands = [
    f"cd esnet-smartnic-fw; ./build.sh",
    f"docker image ls"
]
execute_commands(node, commands)

In [ ]:
# save image to file and upload to the storage VM
# also tar up the sn-stack/ directory and ship to storage VM
commands = [
    f"docker save esnet-smartnic-fw | gzip > esnet-smartnic-fw-{docker_image_version}.tar.gz",
    f"cd esnet-smartnic-fw; tar -zcf sn-stack-{docker_image_version}.tar.gz sn-stack/",
    f"curl -s -k -u {nginx_user}:{nginx_password} -T esnet-smartnic-fw-{docker_image_version}.tar.gz https://{storage_vm_ip}/ejfat-data/smartnic-docker-images/",
    f"cd esnet-smartnic-fw; curl -s -k -u {nginx_user}:{nginx_password} -T sn-stack-{docker_image_version}.tar.gz https://{storage_vm_ip}/ejfat-data/smartnic-docker-images/"
]

execute_commands(node, commands)

## Download containers from storage VM and install

You can use previously built containers here by downloading them from the storage VM and installing into Docker.



In [ ]:
# docker image prefixes
docker_image_prefixes = ['smartnic-dpdk-docker', 'xilinx-labtools-docker', 'esnet-smartnic-fw']

# download and install all available docker images with the right version
commands = []

# we add sn-stack here - it's not a docker image, but just a zipped up tree
# we need to download it, but it doesn't get installed as a docker image
for prefix in docker_image_prefixes + 'sn-stack':
    commands.append(f"curl -s -k -u {nginx_user}:{nginx_password} http://{storage_vm_ip}/ejfat-data/smartnic-docker-images/{prefix}-{docker_image_version}.tar.gz > {prefix}-{docker_image_version}.tar.gz")

execute_commands(node, commands)

In [ ]:
# install the images in the docker on the node. 
# Remember to checkout esnet-smartnic-fw repo (no need to build, just check it out)
commands = [ f"if [ ! -e esnet-smartnic-fw ]; then git clone https://github.com/esnet/esnet-smartnic-fw.git; fi" ]
for prefix in docker_image_prefixes:
    commands.append(f"docker load --input {prefix}-{docker_image_version}.tar.gz")

# untar sn-stack into the previously checked out repo
commands.append(f"tar -zxf sn-stack-{docker_image_version}.tar.gz -C esnet-smartnic-fw/")

execute_commands(node, commands)

## Stand up the stack

Here we bring up the FPGA and UDPLBd on top of it for control plane following these instructions: 
- https://github.com/esnet/esnet-smartnic-fw/blob/main/sn-stack/README.INSTALL.md 

In [ ]:
# configure sn-stack/.env
# You can use `openssl rand -base64 24` to generate tokens
env_file = """
# block-start Added by the FABRIC notebook 
# relies on default already having SN_INGRESS_PORT=844${UNIQUE:-0}
FPGA_PCIE_DEV=0000:1f:00
# enables traefik
COMPOSE_PROFILES=smartnic-mgr-vfio-unlock,smartnic-ingress
SN_HOST=lb-node
SN_CFG_AUTH_TOKEN=HJXaxUD8olOum+KgpWnteKF5VY6AJOV2
SN_P4_AUTH_TOKEN=PzpX96fnitMlDBVJN+v2UlGcXnJ+rURG
SMARTNIC_DPDK_IMAGE_URI=smartnic-dpdk-docker:ubuntu-dev
LABTOOLS_IMAGE_URI=xilinx-labtools-docker:ubuntu-dev
SMARTNIC_FW_IMAGE_URI=esnet-smartnic-fw:ubuntu-dev
# block-end 
"""

# upload the sn-cfg setup file to be executed from inside the container once it is up
sn_cfg_file="config/u280_setup.sh"
# scratch is mounted into the container
sn_cfg_install_path="/home/ubuntu/esnet-smartnic-fw/sn-stack/scratch/u280_setup.sh"
result = node.upload_file(sn_cfg_file, sn_cfg_install_path)

commands = [
    f"echo '{env_file}' >> ~/esnet-smartnic-fw/sn-stack/.env",
    f"chmod a+x {sn_cfg_install_path}"
]
execute_commands(node, commands)

In [ ]:
# bring up the stack

commands = [
    f"cd esnet-smartnic-fw/sn-stack; docker compose up -d"
]
execute_commands(node, commands)

In [ ]:
# wait for some time, check the logs
commands = [
    f"cd esnet-smartnic-fw/sn-stack; docker compose logs smartnic-hw"
]
execute_commands(node, commands)

In [ ]:
# run initial configuration
# the command should print out details about the detected card something like 
# ----------------------------------------
# Device ID: 0
# ----------------------------------------
# PCI:
#     Bus ID:    0000:1f:00.0
#     Vendor ID: 0x10ee
#     Device ID: 0x903f
# Build:
#     Number: 0x0000e154
#     Status: 0x09231435
#     DNA[0]: 0x4cc061c5
#     DNA[1]: 0x016ad0a3
#     DNA[2]: 0x40020000
# Card:
#     Name:                  ALVEO U280 PQ
#     Profile:               U280
#     Serial Number:         21770329D004
#     Revision:              1.0
#     SC Version:            4.0
#     Config Mode:           MASTER_SPI_X4
#     Fan Presence:          P
#     Total Power Available: 225W
#     Cage Types:
#     MAC Addresses:
#         0: 00:0A:35:0E:26:36
#         1: 00:0A:35:0E:26:37
#         2: FF:FF:FF:FF:FF:FF
#         3: FF:FF:FF:FF:FF:FF
# Critically it should show 'Link: up' at the bottom for both ports
#
commands = [
    f"cd esnet-smartnic-fw/sn-stack; docker compose exec smartnic-fw /scratch/u280_setup.sh"
]

execute_commands(node, commands)

In [ ]:
# bring the stack down
commands = [
    f"cd esnet-smartnic-fw/sn-stack;docker compose down -v --remove-orphans"
]
execute_commands(node, commands)

## Stand up UDPLBd Control Plane

## Extend the slice (as needed)

If you need to extend the storage slice, you can just execute the following two cells. They display the slice expiration date and optionally extend by 2 weeeks. 

In [ ]:
slice = fablib.get_slice(name=slice_name)
a = slice.show()
nets = slice.list_networks()
nodes = slice.list_nodes()

Renew the slice

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end host to now plus 14 days
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

## Delete the Slice (as needed)

Please delete your slice when you are done with your experiment.


In [ ]:
slice = fablib.get_slice(name=slice_name)
slice.delete()